# 1. Multivariable Linear Regression

In [4]:
import torch
from torch import optim

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(),
        cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

# 2. Multivariable Linear Regression with nn.Module and nn.functional

In [5]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력차원: 3, 출력차원: 1
    
    def forward(self, x): # Hypothesis 계산
        return self.linear(x)

In [8]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5) # model의 파라미터!

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    Hypothesis = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(Hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 80207.101562
Epoch    1/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 25141.064453
Epoch    2/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 7880.781250
Epoch    3/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 2470.600098
Epoch    4/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 774.795471
Epoch    5/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 243.250900
Epoch    6/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 76.639267
Epoch    7/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 24.415552
Epoch    8/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000]) Cost: 8.045964
Epoch    9/20 hypothesis: tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.10

# 3. Multivariable Linear Regression with Dataset and DataLoader (Final)

In [9]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [13]:
# Dataset
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

dataset = CustomDataset()

In [14]:
# DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True
)

In [15]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력차원: 3, 출력차원: 1
    
    def forward(self, x): # Hypothesis 계산
        return self.linear(x)

In [16]:
# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5) # model의 파라미터!

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch: {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch: 1/3 Cost: 28046.201172
Epoch    0/20 Batch: 2/3 Cost: 19746.660156
Epoch    0/20 Batch: 3/3 Cost: 5667.567383
Epoch    1/20 Batch: 1/3 Cost: 679.330750
Epoch    1/20 Batch: 2/3 Cost: 429.745789
Epoch    1/20 Batch: 3/3 Cost: 93.460770
Epoch    2/20 Batch: 1/3 Cost: 25.248043
Epoch    2/20 Batch: 2/3 Cost: 11.477489
Epoch    2/20 Batch: 3/3 Cost: 1.096107
Epoch    3/20 Batch: 1/3 Cost: 3.764797
Epoch    3/20 Batch: 2/3 Cost: 1.046961
Epoch    3/20 Batch: 3/3 Cost: 1.182909
Epoch    4/20 Batch: 1/3 Cost: 0.413429
Epoch    4/20 Batch: 2/3 Cost: 2.799404
Epoch    4/20 Batch: 3/3 Cost: 0.138232
Epoch    5/20 Batch: 1/3 Cost: 1.906268
Epoch    5/20 Batch: 2/3 Cost: 0.251541
Epoch    5/20 Batch: 3/3 Cost: 3.351644
Epoch    6/20 Batch: 1/3 Cost: 0.884219
Epoch    6/20 Batch: 2/3 Cost: 2.577725
Epoch    6/20 Batch: 3/3 Cost: 0.912874
Epoch    7/20 Batch: 1/3 Cost: 1.444847
Epoch    7/20 Batch: 2/3 Cost: 2.317079
Epoch    7/20 Batch: 3/3 Cost: 0.080316
Epoch    8/20 Batch: 1